<a href="https://colab.research.google.com/github/lszam/brazilianplatform/blob/main/grdSurfer_2_nc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update -q
!apt-get install -y gdal-bin -q

In [35]:
!gdalinfo --version

GDAL 3.6.4, released 2023/04/17


In [36]:
!gdal_translate -of netCDF BR-DTM-REV2024.grd BR-DTM-REV2024.nc

Input file size is 3159, 5707
0...10...20...30...40...50...60...70...80...90...100 - done.


In [37]:
import xarray as xr

# Verificar arquivo de saída
ds = xr.open_dataset('/content/BR-DTM-REV2024.nc')
ds

<xarray.Dataset> Size: 72MB
Dimensions:  (lat: 5707, lon: 3159)
Coordinates:
  * lat      (lat) float64 46kB -4.482e+06 -4.481e+06 ... 1.223e+06 1.224e+06
  * lon      (lon) float64 25kB -5.941e+06 -5.94e+06 ... -2.784e+06 -2.783e+06
Data variables:
    Band1    (lat, lon) float32 72MB ...
Attributes:
    Conventions:  CF-1.5
    GDAL:         GDAL 3.6.4, released 2023/04/17
    history:      Sat Mar 08 14:16:04 2025: GDAL CreateCopy( BR-DTM-REV2024.n...

-------------------------------------------------------
**Para reduzir o tamanho e compactar o arquivo:**

## **Reduzir pelo tipo da variável**

In [38]:
# Reduzir tamanho
# Converter coords de float64 para float32 e reatribuir
ds = ds.assign_coords(lat=ds.coords['lat'].astype('float32'))
ds = ds.assign_coords(lon=ds.coords['lon'].astype('float32'))
ds

<xarray.Dataset> Size: 72MB
Dimensions:  (lat: 5707, lon: 3159)
Coordinates:
  * lat      (lat) float32 23kB -4.482e+06 -4.481e+06 ... 1.223e+06 1.224e+06
  * lon      (lon) float32 13kB -5.941e+06 -5.94e+06 ... -2.784e+06 -2.783e+06
Data variables:
    Band1    (lat, lon) float32 72MB ...
Attributes:
    Conventions:  CF-1.5
    GDAL:         GDAL 3.6.4, released 2023/04/17
    history:      Sat Mar 08 14:16:04 2025: GDAL CreateCopy( BR-DTM-REV2024.n...

In [39]:
!pip install netCDF4 -q

Salvar como netcdf compacto

In [41]:
import os

# Tentar salvar com compactação zlib, forçando o uso do engine "netcdf4"
compressed_file_path = "BR-DTM-REV2024_compressed.nc"

# Aplicar compressão zlib e salvar corretamente
ds.to_netcdf(
    compressed_file_path,
    encoding={"Band1": {"zlib": True, "complevel": 9}}, # 9 é a compressão máxima
    engine="netcdf4"  # Forçando uso do formato que suporta compressão
)

# Verificar o novo tamanho do arquivo
print(f"Tamanho do arquivo original: {os.path.getsize('BR-DTM-REV2024.nc') / (1024 * 1024)}")
print(f"Tamanho do arquivo compactado (compactação nativa do netcdf: zlib): {os.path.getsize(compressed_file_path) / (1024 * 1024)}")

Tamanho do arquivo original: 68.8411750793457
Tamanho do arquivo compactado (compactação nativa do netcdf: zlib): 28.379545211791992


Converter z em inteiro

In [42]:
# Tentar converter a variável principal para int16 e compactar novamente
int16_file_path = "BR-DTM-REV2024_int16.nc"

# ** Primeiro, verificar os valores mínimo e máximo da variável para ver se
# a conversão para int16 é viável
# ** O tipo int16 (inteiro de 16 bits) só pode armazenar valores inteiros
# dentro do intervalo -32.768 a 32.767. Então, se o dataset original em float32
# tiver valores maiores que este intervalo, a conversão pode gerar truncamento
# dos dados
band1_min = ds["Band1"].min().values
band1_max = ds["Band1"].max().values
print ("Valores mínimo e máximo:", band1_min, band1_max)

# Converter apenas se os valores estiverem dentro do intervalo de int16
# Se não der certo, pode tentar o int32

if band1_min >= -32768 and band1_max <= 32767:
    ds_int16 = ds.copy()
    ds_int16["Band1"] = ds_int16["Band1"].astype("int16")

    # Salvar com compressão máxima
    ds_int16.to_netcdf(
        int16_file_path,
        encoding={"Band1": {"zlib": True, "complevel": 9}},
        engine="netcdf4"
    )

    # Verificar o tamanho do arquivo int16 compactado
    int16_size_mb = os.path.getsize(int16_file_path) / (1024 * 1024)
else:
    int16_size_mb = None  # Não foi possível converter para int16

print(f"Tamanho do arquivo original: {os.path.getsize('BR-DTM-REV2024.nc') / (1024 * 1024)}")
print(f"Tamanho do arquivo compactado (compactação nativa do netcdf: zlib): {os.path.getsize(compressed_file_path) / (1024 * 1024)}")
print(f"Tamanho do arquivo int16 (compactação zlib): {int16_size_mb}")


Valores mínimo e máximo: -6496.3203125 163.3430938720703


/usr/local/lib/python3.11/dist-packages/xarray/core/duck_array_ops.py:236: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)


Tamanho do arquivo original: 68.8411750793457
Tamanho do arquivo compactado (compactação nativa do netcdf: zlib): 28.379545211791992
Tamanho do arquivo int16 (compactação zlib): 11.05407428741455


---------------------------------------------------------------------
## **Compressão usando rar**

In [43]:
!sudo apt update && sudo apt install rar -y

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,533 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,988 kB]
Fetched 4,778 kB in 2s (2,658 kB/s)
Reading package lists... Done
Building dependency tree... Don

In [44]:
!ls -lh /content/BR-DTM-REV2024_int16.nc

-rw-r--r-- 1 root root 12M Mar  8 15:09 /content/BR-DTM-REV2024_int16.nc


In [45]:
!rar a /content/BR-DTM-REV2024_int16.rar /content/BR-DTM-REV2024_int16.nc


RAR 5.50   Copyright (c) 1993-2017 Alexander Roshal   11 Aug 2017
Trial version             Type 'rar -?' for help

Evaluation copy. Please register.

Creating archive /content/BR-DTM-REV2024_int16.rar

Adding    /content/BR-DTM-REV2024_int16.nc                                36% 72%100%  OK 
Done


In [46]:
!ls -lh /content/BR-DTM-REV2024_int16.rar

-rw-r--r-- 1 root root 11M Mar  8 15:10 /content/BR-DTM-REV2024_int16.rar


## **Download do arquivo**

In [48]:
# Download do arquivo .nc de saída
from google.colab import files
files.download('/content/BR-DTM-REV2024_int16.nc')
files.download('/content/BR-DTM-REV2024_int16.rar')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>